

### Loading the data.

The data is in json format,so the first step inolves in converting the .json to pandas dataframe.


In [111]:
import numpy as np
import pandas as pd
import json
import gzip

#Converting into panda's dataframe
yesterday=pd.read_json('C:/Users/Srikanth/Desktop/DatraWeave/yesterday.json.gz_out',lines='true')
yesterday.shape   # It can be observed that there are almost 1.42 lakhs rows(records)

(141699, 9)

In [112]:
Id=range(141699)
yesterday['Id']=Id
yesterday.head()
# Adding an additional column "Id" for my convenience.

,available_price,category,crawl_date,http_status,mrp,pack_size,subcategory,title,urlh,Id
0,5.99,Household,20180726,200,0.0,30 ct,Trash Bags & Liners,Seventh Generation 13 Gallon Extra Strong Flap...,1ef0f41ebcee030a35440405f2f8b0165c719c89,0
1,5.49,Household,20180726,200,0.0,25 ct,Trash Bags & Liners,BioBag Food Scrap Bags Small 3 Gallon - 25 CT ...,c566570abdbe95b5e6a78eb8b3b2424108bf36a2,1
2,5.69,Household,20180726,200,0.0,30.0 ct,Trash Bags & Liners,If You Care Certified Compostable Food Waste B...,3b22b28b351dec75664b17fcf586023ddc21704c,2
3,5.99,Household,20180726,200,0.0,12 ct,Trash Bags & Liners,BioBag Food Scrap Bags Tall Kitchen 13 Gallon ...,7ff787f49e137237054f95e7cb5b0f8611e7f6fb,3
4,5.99,Household,20180726,200,0.0,20 ct,Trash Bags & Liners,Seventh Generation 13 Gal with Drawstring Clos...,a7ad16e71257855c033aad46eb5082a2153b97c6,4


In [113]:
today=pd.read_json('C:/Users/Srikanth/Desktop/DatraWeave/today.json.gz_out',lines='true')
today.shape # Alomst 1.423 lakhs records, a bit more than yesterday.

(142291, 9)

In [114]:
Id=range(142291)
today['Id']=Id
today.head()

,available_price,category,crawl_date,http_status,mrp,pack_size,subcategory,title,urlh,Id
0,5.99,Household,20180727,200,0.0,30 ct,Trash Bags & Liners,Seventh Generation 13 Gallon Extra Strong Flap...,1ef0f41ebcee030a35440405f2f8b0165c719c89,0
1,5.49,Household,20180727,200,0.0,25 ct,Trash Bags & Liners,BioBag Food Scrap Bags Small 3 Gallon - 25 CT ...,c566570abdbe95b5e6a78eb8b3b2424108bf36a2,1
2,5.69,Household,20180727,200,0.0,30.0 ct,Trash Bags & Liners,If You Care Certified Compostable Food Waste B...,3b22b28b351dec75664b17fcf586023ddc21704c,2
3,5.99,Household,20180727,200,0.0,12 ct,Trash Bags & Liners,BioBag Food Scrap Bags Tall Kitchen 13 Gallon ...,7ff787f49e137237054f95e7cb5b0f8611e7f6fb,3
4,5.99,Household,20180727,200,0.0,20 ct,Trash Bags & Liners,Seventh Generation 13 Gal with Drawstring Clos...,a7ad16e71257855c033aad46eb5082a2153b97c6,4


In [115]:
a=today
b=yesterday
#Assigning the original data(yesterday and today) to "a" and "b" since it is not wise to work on original data.

### 1.Number of Urls which are overlapping.
By the term overlapping I assume that, the no. of rows with urls which are common in both dataframes.

In [6]:
intersection=list(set(a['urlh']).intersection(set(b['urlh']))) # All the urlhs in both today and yesterday's data.
#Here i did set intersection and converted it into list and found the length.
intersection[0:4]
#These are the urlhs which are present in both dataframes.

['323a8894106bad1f43402d0f6e0aa5feb49635d4',
 '503020b52b0ffa31e593a127e359617392841b95',
 '8a256673e44ebdac0d6c6b5d702118f0313ce91c',
 'df6a05434334b6017bca2ba14ed96e2af1b71e35']

In [7]:
len(intersection)
# 129673 records have overlapping

129673

In [8]:
overlap_today=[dict(a.loc[i]) for i in range(len(a)) if a['urlh'][i] in intersection]
#Here we find the records which are having urlhs in intersection.

In [9]:
len(overlap_today)
#Out of 142291 records in today, 140141 records have urlhs which are present in yesterday.

140414

In [10]:
overlap_yesterday=[dict(b.ix[i]) for i in range(len(b)) if b['urlh'][i] in intersection]
#Out of 141699, 140413 records have urlhs intersection

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [11]:
len(overlap_yesterday)

140413

In [12]:
overlap_yesterday[1]
#It is a list of dicts, so 1st index of this list is same as the first row of dataframe.

{'available_price': 5.49,
 'category': 'Household',
 'crawl_date': 20180726,
 'http_status': 200,
 'mrp': 0.0,
 'pack_size': '25 ct',
 'subcategory': 'Trash Bags & Liners',
 'title': 'BioBag Food Scrap Bags Small 3 Gallon - 25 CT - 25 ct',
 'urlh': 'c566570abdbe95b5e6a78eb8b3b2424108bf36a2',
 'Id': 1}

In [13]:
overlap_today1=pd.DataFrame.from_records(overlap_today)
#Here i converted the list of dicts to dataframe for easy understanding.

### 2. Calculate price difference.

In [398]:
yesterday_dup=[]
for i in range(len(overlap_yesterday)):
    if overlap_yesterday[i]['urlh']==overlap_yesterday[i]['urlh'] and overlap_yesterday[i]['http_status']==200:
        yesterday_dup.append(overlap_yesterday[i])
# In 3rd line, all the urls with http=200 are taken and appended in the list named yesterday_dup. 
#This list will contain duplicate urls. So in next step we will drop duplicates.

In [399]:
len(yesterday_dup)
# It can be observed that therse are only 8 duplicate urls in yesterday's which has http-value other than 200.
# But point to be kept in mind is that there could be lot of duplicate urls with http value=200.

140405

In [400]:
overlap_yesterday1=pd.DataFrame.from_records(yesterday_dup)
#Here i converted the list of dicts to dataframe for easy understanding.

In [401]:
overlap_dup1=overlap_yesterday1.drop_duplicates(subset={"urlh"}, keep='first', inplace=False)
overlap_dup1.shape
#here it can be observed that after dropping duplicate urlhs with http=200, the yesterday's records 
# have dropped to 129673

(129673, 10)

In [402]:
today_dup=[]
for i in range(len(overlap_today)):
    if overlap_today[i]['urlh']==overlap_today[i]['urlh'] and overlap_today[i]['http_status']==200:
        today_dup.append(overlap_today[i])

In [403]:
len(today_dup)

140407

In [404]:
overlap_today1=pd.DataFrame.from_records(today_dup)
#Here i converted the list of dicts to dataframe for easy understanding.

In [405]:
overlap_dup2=overlap_today1.drop_duplicates(subset={"urlh"}, keep='first', inplace=False)
overlap_dup2.shape
#here it can be observed that after dropping duplicate urlhs with http=200, the yesterday's records 
# have dropped to 129673

(129673, 10)

In [406]:
price_difference=overlap_dup2['available_price']-overlap_dup1['available_price']

In [407]:
price_difference[::-1]
#Here i printed the list in reverse order since price difference is observed in last records not in the staring records.

140404      NaN
140403      NaN
140402     2.01
140401     0.00
140400    20.99
140399     0.00
140398   -30.00
140397    -1.00
140396    -1.00
140395     2.00
140394     1.00
140393    -2.00
140392     6.00
140391     2.00
140390     4.00
140389     5.20
140388     6.00
140387    -6.20
140386   -10.00
140385     7.50
140384     2.50
140383    -6.50
140382     0.50
140381     3.00
140380    -1.00
140379    -0.50
140378     5.00
140377     0.51
140376   -13.00
140375     1.99
          ...  
29         0.00
28         0.00
27         0.00
26         0.00
25         0.00
24         0.00
23         0.00
22         0.00
21         0.00
20         0.00
19         0.00
18         0.00
17         0.00
16         0.00
15         0.00
14         0.00
13         0.00
12         0.00
11         0.00
10         0.00
9          0.00
8          0.00
7          0.00
6          0.00
5          0.00
4          0.00
3          0.00
2          0.00
1          0.00
0          0.00
Name: available_price, L

In [241]:
def diff(s):
"""Here we are gettin the index where the available_price is not same"""
    index=[]
    for i in s.index:
        if s[i]!=0:
            index.append(i)
    return(index)
aa=diff(price_difference)
#Here we are getting the indexes where the available_price is not zero(prices are different)

In [244]:
len(aa)

82918

### 3. Unique Categories in both files.

In [27]:
count1=[]
unique1=today['category'].unique()
for i in unique1:
    count1.append(i)
len(count1)
#There are 50 unique categories in today dataframe

50

In [28]:
count_tday_unique=today.category.unique()

#These are the unique values, if we count_tday_unique.size...we can get the size too but still I wrote/
#the code to get the unique values in the above cell.

count_tday_unique=list(count_tday_unique)# Conver the array to list.


In [410]:
count2=[]
unique2=yesterday['category'].unique()
for i in unique2:
    count2.append(i)
len(count2)
#There are 50 unique categories in today dataframe

50

### 4. List of Categories not overlapping

In [411]:
%%time
not_overlap=[]
i=0
while i< len(b): #Here we chose lenght of b cuz records in yesterday is lesser than today.
    if a['category'][i] != b['category'][i]:
        not_overlap.append(a['category'][i])
    i+=1

Wall time: 2.68 s


In [412]:
len(not_overlap)

19577

### 5. Generate the stats with count for all taxonomies.

In [371]:
#group = a.groupby('category')

#df2 = group.apply(lambda x: x['subcategory'])
#Here we will get a dataframe where we will get All the 50 Categories and their corresponding Subcategories.

In [18]:
#today_tex=[]
#for i in count_tday_unique[]:
#    if a['category'][i]==i:
 #       today_tex.append(a['subcategory'][i])

In [16]:
df_tax=a[['category', 'subcategory']]

In [25]:
df_tax[:10]

,category,subcategory
0,Household,Trash Bags & Liners
1,Household,Trash Bags & Liners
2,Household,Trash Bags & Liners
3,Household,Trash Bags & Liners
4,Household,Trash Bags & Liners
5,Household,Paper Goods
6,Household,Paper Goods
7,Household,Paper Goods
8,Household,Paper Goods
9,Household,Paper Goods


In [123]:
tax_dict={k: v["category"].tolist() for k,v in df_tax.groupby("subcategory")}
tax_dict['Silver Rum'][:6]
#Here in this cell we get dicts where the unique categories will be the keys 
#and the corresponding subcategories will be inthe list as values to the key.

['Rum', 'Rum', 'Rum', 'Rum', 'Rum', 'Rum']

In [84]:
for myKey in tax_dict.keys():
    print(myKey,'<', Counter(list(tax_dict[myKey])))   

    ##Here we get the key '<' Subcategory: count

Adult Care < Counter({'Personal Care': 67})
Aged Rum < Counter({'Rum': 70})
Air Fresheners & Candles < Counter({'Household': 809, 'Personal Care': 94})
Albarino < Counter({'White Wine': 10})
Allergy < Counter({'Allergy & Cold Essentials': 52})
Amber/Red Ale < Counter({'Ale Beer': 63})
Amber/Red Lager < Counter({'Lager Beer': 18})
American All-Malt Lager < Counter({'Lager Beer': 16})
American Wild Ale < Counter({'Specialty Style Beer': 8})
American-Style Lager < Counter({'Lager Beer': 110})
Anejo < Counter({'Tequila': 57})
Aperitif < Counter({'Dessert & Fortified Wine': 25})
Apple Cider < Counter({'Cider': 118})
Armagnac < Counter({'Brandy & Cognac': 7})
Aromatherapy < Counter({'Personal Care': 34})
Asian Foods < Counter({'International': 945})
Baby Accessories < Counter({'Babies': 253, 'Baby & Child': 19})
Baby Bath & Body Care < Counter({'Babies': 285, 'Baby & Child': 66})
Baby First Aid & Vitamins < Counter({'Babies': 197, 'Baby & Child': 47})
Baby Food & Formula < Counter({'Babies':

### 6. Generate a new file where mrp is normalized.

In [ ]:
%%time
a
for i in range(len(a)): # Here we are taking today's data
    if a['mrp'][i]==0 or a['mrp'][i]=='nan' :
        a['mrp'][i]='NA'

# isinstance(value,float)=True means values in float.
# If mrp value is 'na' or non-float or 0 then, it has been set to 'NA'

##### The above code was taking awefully long time to execute. But the code is fine and working.

In [133]:
with open('a11.json', 'w') as f:
    f.write(a.to_json(orient='records', lines=True))

## Thank YOU

This assignment was really fun. You explicitly mentioned in 6th point to have fun. Yes I had fun, learnt a a lot doing this assignment.

My understanding in ML is good, can't wait to share my knowledge with you people.
Thank You.